# Recommender System Modeling
In this notebook, we will work through multiple iterations of our movie recommender system model. First, we will create a simpler model using just the movie reviews from IMDb. This will give us a baseline model that we can potentially improve on by adding more detailed features.

In [1]:
# Imports
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

## Simple Recommender System
This is a simple recommender system using cosine similarity that can be improved upon and replaced when all our data is properly formatted.

In [2]:
# read in merged review data
merged_reviews = pd.read_csv('../Data/Large-Data/merged_reviews.csv')
merged_reviews

/var/folders/jc/vvd13n2d3m33dzsrsg9ydhb00000gn/T/ipykernel_926/2414319021.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_reviews = pd.read_csv('../Data/Large-Data/merged_reviews.csv')


,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain
...,...,...,...,...
29326187,ur0581842,tt0107977,6.0,Robin Hood: Men in Tights
29326188,ur3174947,tt0103776,8.0,Batman Returns
29326189,ur4592639,tt0107423,9.0,Lipstick on Your Collar
29326190,ur4581944,tt0102614,8.0,Out for Justice


In [3]:
pivot = merged_reviews.pivot_table(values = 'scaled_rating', index = 'title', columns = 'user_id')

pivot.head()

/var/folders/jc/vvd13n2d3m33dzsrsg9ydhb00000gn/T/ipykernel_926/1184987666.py:1: PerformanceWarning: The following operation may generate 350618041382 cells in the resulting pandas object.
  pivot = merged_reviews.pivot_table(values = 'scaled_rating', index = 'title', columns = 'user_id')


: 

In [ ]:
pivot.shape

In [ ]:
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

In [ ]:
distances = pairwise_distances(pivot_sparse, metric = 'cosine')

In [ ]:
similarities = 1.0 - distances

In [ ]:
recommender = pd.DataFrame(similarities, index = pivot.index, columns=pivot.index)

recommender.head()